In [1]:
import sys
import pandas as pd
import numpy as np
import math
from scipy.spatial.distance import cosine
import xml.etree.ElementTree as ET
import pickle
import re

from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
from nltk.corpus import stopwords

stopwords.words('arabic')

['إذ',
 'إذا',
 'إذما',
 'إذن',
 'أف',
 'أقل',
 'أكثر',
 'ألا',
 'إلا',
 'التي',
 'الذي',
 'الذين',
 'اللاتي',
 'اللائي',
 'اللتان',
 'اللتيا',
 'اللتين',
 'اللذان',
 'اللذين',
 'اللواتي',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'أم',
 'أما',
 'أما',
 'إما',
 'أن',
 'إن',
 'إنا',
 'أنا',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'إنما',
 'إنه',
 'أنى',
 'أنى',
 'آه',
 'آها',
 'أو',
 'أولاء',
 'أولئك',
 'أوه',
 'آي',
 'أي',
 'أيها',
 'إي',
 'أين',
 'أين',
 'أينما',
 'إيه',
 'بخ',
 'بس',
 'بعد',
 'بعض',
 'بك',
 'بكم',
 'بكم',
 'بكما',
 'بكن',
 'بل',
 'بلى',
 'بما',
 'بماذا',
 'بمن',
 'بنا',
 'به',
 'بها',
 'بهم',
 'بهما',
 'بهن',
 'بي',
 'بين',
 'بيد',
 'تلك',
 'تلكم',
 'تلكما',
 'ته',
 'تي',
 'تين',
 'تينك',
 'ثم',
 'ثمة',
 'حاشا',
 'حبذا',
 'حتى',
 'حيث',
 'حيثما',
 'حين',
 'خلا',
 'دون',
 'ذا',
 'ذات',
 'ذاك',
 'ذان',
 'ذانك',
 'ذلك',
 'ذلكم',
 'ذلكما',
 'ذلكن',
 'ذه',
 'ذو',
 'ذوا',
 'ذواتا',
 'ذواتي',
 'ذي',
 'ذين',
 'ذينك',
 'ريث',
 'سوف',
 'سوى',
 'شتان',
 'عدا',
 'عسى',
 'عل'

In [4]:
sys.path.insert(0, '../EVAL/scorer_v2.3/MAP_scripts/')
from ev import evaluate

In [5]:
def readXML(path):
    """
    Read XML file into a dictionary
    """
    tree = ET.parse(path)
    root = tree.getroot()
    
    dataset = pd.DataFrame(columns=['QID', 'QAID'], dtype=int)
    
    for Question in root:
        QID = int(Question.get('QID'))
        Qtext = Question.find('Qtext').text
        
        for QApair in Question.iter('QApair'): 
            QAID = int(QApair.get('QAID'))
            QArel = QApair.get('QArel')
            QAconf = QApair.get('QAconf')
            QAquestion = QApair.find('QAquestion').text
            QAanswer = QApair.find('QAanswer').text
            
            dataset = dataset.append({'QID': QID,
                                    'QAID': QAID,
                                    'Qtext': Qtext,
                                    'QAquestion': QAquestion,
                                    'QAanswer': QAanswer,
                                    'QArel': 0 if QArel == 'I' else 1,
                                    'QAconf': QAconf}, ignore_index=True)
            
            with open('../TEST/2017/PROCESSED/' + str(QID) + '/' + str(QID) + '_q.lemma', encoding='utf-8') as f:
                Qtext_lemmatized = ''.join(f.readlines())
                Qtext_lemmatized = ' '.join([lemma.strip() for lemma in re.findall(r'Lemma:(.*?)BW:', Qtext_lemmatized)])
                
            with open('../TEST/2017/PROCESSED/' + str(QID) + '/' + str(QAID) + '_qaq.lemma', encoding='utf-8') as f:
                QAquestion_lemmatized = ''.join(f.readlines())
                QAquestion_lemmatized = ' '.join([lemma.strip() for lemma in re.findall(r'Lemma:(.*?)BW:', QAquestion_lemmatized)])
            
            with open('../TEST/2017/PROCESSED/' + str(QID) + '/' + str(QAID) + '_qaq.lemma', encoding='utf-8') as f:
                QAanswer_lemmatized = ''.join(f.readlines())
                QAanswer_lemmatized = ' '.join([lemma.strip() for lemma in re.findall(r'Lemma:(.*?)BW:', QAanswer_lemmatized)])
                
            dataset_lemmatized = dataset.append({'QID': QID,
                                    'QAID': QAID,
                                    'Qtext': Qtext_lemmatized,
                                    'QAquestion': QAquestion_lemmatized,
                                    'QAanswer': QAanswer_lemmatized,
                                    'QArel': 0 if QArel == 'I' else 1,
                                    'QAconf': QAconf}, ignore_index=True)
            
    dataset.set_index(['QID', 'QAID'], inplace=True)
    return dataset, dataset_lemmatized

In [6]:
test_dataset, lemmatized_test_dataset = readXML('../TEST/2017/SemEval2017-Task3-CQA-MD-test.xml')

In [7]:
test_dataset = test_dataset.sort_index(level=0, ascending=[False, True])
test_dataset = test_dataset.reset_index().drop_duplicates().set_index(['QID', 'QAID'])

lemmatized_test_dataset = lemmatized_test_dataset.sort_index(level=0, ascending=[False, True])
lemmatized_test_dataset = lemmatized_test_dataset.reset_index().drop_duplicates().set_index(['QID', 'QAID'])

In [8]:
from nltk import wordpunct_tokenize
from nltk.stem import ISRIStemmer

class StemTokenizer(object):
    def __init__(self):
        self.wnl = ISRIStemmer()
    def __call__(self, doc):
         return [self.wnl.stem(t) for t in wordpunct_tokenize(doc)]

In [11]:
model = Pipeline([('tfidf', TfidfVectorizer(min_df=1, max_df=0.1, tokenizer=StemTokenizer(), stop_words=stopwords.words('arabic'), smooth_idf=False, sublinear_tf=True, norm='l2', max_features=10000)),
                ('classifier', GradientBoostingClassifier(10))])
model.fit(test_dataset['Qtext'])

ValueError: bad input shape ()

In [641]:
scores = model.predict()

In [642]:
test_dataset['score'] = [0 if similarity < 0 or math.isnan(similarity) else round(similarity,4) for similarity in similarities ]

In [643]:
test_dataset['relevance'] = ['true' if similarity > 0.5 else 'false' for similarity in similarities]

In [644]:
test_dataset['rank'] = 0

In [645]:
test_dataset.to_csv('../EVAL/SemEval2017-Task3-CQA-MD-test-lsa-mlp.xml.pred', sep='\t', header=None, columns=['QID', 'QAID', 'rank', 'score', 'relevance' ])

c:\python35\lib\site-packages\pandas\core\indexing.py:972: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_nested_tuple(tup)


In [646]:
test_dataset = test_dataset.sort_index(level=0, ascending=[False, True])
test_dataset = test_dataset.reset_index().drop_duplicates().set_index(['QID', 'QAID'])
test_dataset

QAanswer QAconf  \
QID   QAID                                                               
69579 43523   يعود تطبل البطن أو ما يعرف بغازات البطن الى ال...   None   
      51753   قبل أن تبدأ ببرنامج المشي، يتطلب منك أن تعرف ف...   None   
      61846   التهاب الدم أو ما يعرف طبيا بتعفن الدم هو أحد ...   None   
      63972   التهاب الكلى هو أمر شائع لدى مرضى الذئبة و يحد...   None   
      117912  الأول هو اخْتِبار أَضْدادِ الحالَّةِ العُقْدِي...   None   
      229172  العصب بنقل الايعازات الحسية والحركية من الجهاز...   None   
      289894  لا ينصح باستخدام الكورتيزون بهدف زيادة الوزن ل...   None   
      684584  غالبا هذا سببه الرهاب الاجتماعي و رغم ذلك بجب ...   None   
      880238           نبدأ بعلاج دوائي اذا لم ينفع بنعمل الكي   None   
69578 47192   ممكن ان يكون شئ طبيعي لكن للاطمئنان انصحك بمرا...   None   
      52302   نرجو عمل الفحوصات التالية لتحديد طبيعة عمل الم...   None   
      86270   انصحك باجراء فحص لنسبه هرمون الحمل بالدم. واذا...   None   
      304444                                  يجب عمل فحص الحمل   None   
      432499  يجب انتظار الدورة اذا لم تاتى فيمكن عمل تحليل ...   None   
      561870       تحدث مثل هذه الاعراض مع الدورة ولاشىء فى ذلك   None   
      696681  بعد الاحهاض او الولاده وارد ان تتاخر الدوره لا...   None   
      756956  خط واحد يعني ما في حمل انتطري ختنزل الدوره لو ...   None   
      875015  الدورة غير منتظمة ولا يمكن الحساب عليها ويجب م...   None   
69577 22960   غازات وانتفاخ البطن اسبابه عديده من أبرزها الا...   None   
      28732   هذه الأعراض تظهر عادة في وقت واحد في حالات الق...   None   
      37830   عليك مراجعة اختصاصي الجراحة العامة لتقييم الفت...   None   
      53616   لا داعي للقلق، ولكن الخطوة الأولى تتركز على مح...   None   
      65280   الدوخة أو الدوار هما إحساس المريض بدوران الجسم...   None   
      108734  يجب فحصك من قبل طبيب لمعرفة سبب الالم ان كان ف...   None   
      131938  رغم أنك ذكرت أن فحوصات القلب سليمة لم تتطرق إل...   None   
      293946  لا بد من فحص الطبي مباشر للكشف عن العلامات الس...   None   
      836838  جرثومة المعده.... لا تسبب هذه الاعراض(امساك/صد...   None   
69576 8978    مغاطس ماء فاتر مع البرمنغنات. أو مع البابونج.و...   None   
      56635   معظم الشقوق لا تحتاج لعلاج وقد تختفي الأعراض خ...   None   
      66564   بما انها لا تسبب الم او نزف.....فالمطلوب هو ان...   None   
...                                                         ...    ...   
66718 364491  الأعراض التي ذكرتها غالبا ماتكون بسبب النبضات ...   None   
      372893  الأخت الفاضلة! العكس هو الصحيح , فان الوسواس و...   None   
      720920  يجب اولا فحص ما قلت انه تحجر من قبل طبيب جراح ...   None   
66715 42543   :لا يوجد علاج شافي للحساسية وإنما علاجات تخفف ...   None   
      54742   هذه الحبوب هي حب شباب ،أسباب ظهور حب الشباب يت...   None   
      56706   أسباب ظهور حب الشباب يتعلق بحدوث تغيرات عمرية ...   None   
      58177   هي حبوب دهنية بسبب زيادة افراز الغدد الدهنية ا...   None   
      68780   أولا يجب علاج حب الشباب وبعدها يب\أ علاج البثو...   None   
      70595   هذه الحبوب هي حب شبابأسباب ظهور حب الشباب يتعل...   None   
      218846  بالنسبة للأشخاص الذين يعانون من حب الشباب تقوم...   None   
      593669           اقراص الرواكتان بعد عمل التحاليل اللازمه   None   
      829322  استخدمي كريم يحتوي كورتيكوستيرويد ومضاد حيوي ,...   None   
66713 24877   هو انخفاض معدل ضربات القلب إلى ما دون 60 نبضة ...   None   
      31036                     نعم تعتبر طبيعية ولا داعي للقلق   None   
      38495   لأن النِّتاج القَلْبِي يزداد مع زيادة حجم الجس...   None   
      58127   الموت مفارقة الحياة للجسم، وهو تعبير عن توقف ل...   None   
      93012   هكذا نبض إذا لم يرفقه أعراض يعتبر طبيعي خاصة ع...   None   
      182904  نعم توجد علاقة وغقاً لحدتها نقصاً أو زيادة أما...   None   
      309036      حسب العمر والجنس ومتوسطها يتراوح بين 60 ومائة   None   
      352144  بداية ان معدل نبضات القلب الطبيعي هو من 60-100...   None   
      648111  معدل ضربات القلب تختلف عند الانسان حسب العمر ح...   None   
66711 46944   لزيادة الوزن لابد من تناول طعام اكثر مما يتم 

In [647]:
MAP, Accuracy, P, R, F1  = evaluate('../EVAL/SemEval2017-Task3-CQA-MD-test.xml.subtaskD.relevancy', '../EVAL/SemEval2017-Task3-CQA-MD-test-lsa-mlp.xml.pred')

In [648]:
a

MAP: 0.6136
Accuracy: 0.6215
Precision: 0.8295
Recall: 0.0444
F1: 0.0842
